In [ ]:
# install dependencies if not exists
!pip3 install tensorflow==2.3

In [1]:
# import modules
import tensorflow
from tensorflow import keras 
import logger 
import os
print(tensorflow.__version__)

2.3.0


In [37]:
# prepare model

# naive model with fully connected
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(128,128,3)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1572896   
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 66        
Total params: 1,572,962
Trainable params: 1,572,962
Non-trainable params: 0
_________________________________________________________________


In [26]:
# configs
img_height = 128
img_width  = 128
batch_size = 32

# preprocess training data
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=1/3)
train_set = train_datagen.flow_from_directory(
    './faces',
    seed=10, # TODO: Change to time()
    target_size=(128,128),
    class_mode='categorical', 
    batch_size=32, 
    subset="training"
)

test_set = train_datagen.flow_from_directory(
    './faces',
    seed=10, # TODO: Change to time()
    target_size=(128,128),
    class_mode='categorical', 
    batch_size=32, 
    subset="validation"
)

Found 450 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [46]:
# train 
model.fit_generator(
    train_set, # data to train, a format of (X, y)
    steps_per_epoch=len(train_set), # The number of batch iterations before a training epoch is considered finished. Ignore if whole data is load, in our case, we need this in order to iterate over batches; hence make sure that generator can generate at least: steps_per_epoch * epochs batches
    epochs=50, # number of epochs, i.e number of time that iteration of updating weights goes
    validation_data=test_set, # validation
    validation_steps=len(test_set)
)

Epoch 1/10
15/15 [==============================] - 1s 66ms/step - loss: 0.2320 - accuracy: 0.9667 - val_loss: 0.2786 - val_accuracy: 0.9196
Epoch 2/10
15/15 [==============================] - 1s 37ms/step - loss: 0.2366 - accuracy: 0.9667 - val_loss: 0.3210 - val_accuracy: 0.8795
Epoch 3/10
15/15 [==============================] - 1s 38ms/step - loss: 0.2416 - accuracy: 0.9467 - val_loss: 0.2996 - val_accuracy: 0.8973
Epoch 4/10
15/15 [==============================] - 1s 36ms/step - loss: 0.2171 - accuracy: 0.9733 - val_loss: 0.2919 - val_accuracy: 0.8929
Epoch 5/10
15/15 [==============================] - 1s 45ms/step - loss: 0.2169 - accuracy: 0.9711 - val_loss: 0.2778 - val_accuracy: 0.9196
Epoch 6/10
15/15 [==============================] - 1s 38ms/step - loss: 0.2137 - accuracy: 0.9733 - val_loss: 0.3309 - val_accuracy: 0.8839
Epoch 7/10
15/15 [==============================] - 1s 34ms/step - loss: 0.2009 - accuracy: 0.9800 - val_loss: 0.2686 - val_accuracy: 0.9241
Epoch 8/10
15